In [1]:
import QUANTAXIS as QA

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.1.0
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##``````

# 在这里我们演示一下 下单/交易/结算的整个流程

我们首先会建立一个账户类和一个回测类


In [2]:
 # 初始化一个account
Account=QA.QA_Account()

# 初始化一个回测类
B = QA.QA_BacktestBroker()


在第一天的时候,全仓买入 000001

In [3]:
# 全仓买入'000001'

Order=Account.send_order(code='000001',
                        price=11,
                        money=0.04*Account.cash_available,
                        time='2018-05-09',
                        towards=QA.ORDER_DIRECTION.BUY,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_MONEY
                        )




In [4]:

## 打印order的占用资金
print('ORDER的占用资金: {}'.format((Order.amount*Order.price)*(1+Account.commission_coeff)))

ORDER的占用资金: 39609.9


In [5]:
# 账户剩余资金
print('账户剩余资金 :{}'.format(Account.cash_available))

账户剩余资金 :960000.0


In [6]:
Account.hold

Series([], Name: amount, dtype: float64)

In [7]:
Account.init_hold

Series([], Name: amount, dtype: float64)

In [8]:
Account.hold_available

Series([], Name: amount, dtype: float64)

此时的账户cash并未减少,因为此过程为申报订单(已委托 未成交状态)

回测类接受订单 并返回订单(如果创建成功)

In [9]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

['Acc_z65rKWuO', '2018-05-09 09:31:00', '2018-05-09 00:00:00', '000001', None, 1, 10.96, 10.955, 'trade_success', 3600, 3600, 39447.8595, 0, 'Order_y6KqNdRr', 'Trade_rGShmCpU']
< QA_Order realorder_id Order_y6KqNdRr datetime:2018-05-09 09:30:00 code:000001 amount:3600 price:10.955 towards:1 btype:stock_cn order_id:Order_y6KqNdRr account:Acc_z65rKWuO status:queued >


In [10]:
#B.query_orders(rec_mes.account_cookie,rec_mes.realorder_id)

In [11]:
import pandas as pd

In [12]:
B.query_orders(Account.account_cookie)

,,order_time,code,name,towards,trade_price,order_price,status,order_amount,trade_amount,cancel_amount
account_cookie,realorder_id,,,,,,,,,,
Acc_z65rKWuO,Order_y6KqNdRr,2018-05-09 09:31:00,000001,None,1,10.96,10.955,trade_success,3600,3600,0


In [13]:
#pd.DataFrame(list(B.deal_message.values()),columns=B.orderstatus_headers).set_index(['account_cookie','realorder_id'])

In [14]:
B.query_orders(Account.account_cookie,'filled')

,,trade_time,code,name,towards,trade_amount,trade_price,trade_money,trade_id
account_cookie,realorder_id,,,,,,,,
Acc_z65rKWuO,Order_y6KqNdRr,2018-05-09 00:00:00,000001,None,1,3600,10.96,39447.8595,Trade_8oSy6TFV


In [15]:
trade_mes=B.query_orders(Account.account_cookie,'filled')
res=trade_mes.loc[Order.account_cookie,Order.realorder_id]
Order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)

receive deal


In [16]:
#Order.trade((trade_id, trade_price, trade_amount, trade_time))

账户类接收到回测返回的回报信息,更新账户

In [17]:
Account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax
0,2018-05-09 00:00:00,000001,10.96,3600,960474.952,Order_y6KqNdRr,Order_y6KqNdRr,Trade_8oSy6TFV,Acc_z65rKWuO,9.864,59.184


此时我们可以打印一下现在的状态(现在的状态可以理解为在交易时 买入一只000001股票,但当天尚未收盘)

In [18]:
print('账户的可用资金 {}'.format(Account.cash_available))

账户的可用资金 960474.952


In [19]:
Account.hold

code
000001    3600
Name: amount, dtype: int64

In [20]:
Account.hold_available

code
000001    3600
Name: amount, dtype: int64

In [21]:
Account.init_hold.index_name='code'

In [22]:
import pandas as pd

In [23]:
pd.concat([Account.hold_available,Account.init_hold])

code
000001    3600
Name: amount, dtype: int64

In [24]:
+Account.hold_available

code
000001    3600
Name: amount, dtype: int64

我们注意到 当最初申报订单的时候,可用资金只有950.2999999999302元,而买入成功后,可用资金有3339.9289999998837元,原因是下单的时候模式是市价单模式(QA.ORDER_MODEL.MARKET),故实际成交金额为10.96元

买入以后 账户的持仓为90800股 000001

In [25]:
Account.hold

code
000001    3600
Name: amount, dtype: int64

买入后账户现金表被扩展

In [26]:
Account.cash

[1000000, 960474.952]

因为是t+1的A股市场,故此时可卖数量为0

In [27]:
Account.sell_available

Series([], Name: amount, dtype: float64)

# 执行结算

In [28]:
Account.settle()

# 结算后

In [29]:
Account.cash

[1000000, 960474.952]

In [30]:
Account.cash_available

960474.952

In [31]:
Account.sell_available

code
000001    3600
Name: amount, dtype: int64

In [32]:
Account.hold

code
000001    3600
Name: amount, dtype: int64

# 执行卖出操作

现在的持仓为: 000001 90800股

In [33]:
holdnum=Account.sell_available.get('000001',0)

In [34]:
holdnum

3600

申报一个卖出单,把可卖全部卖出

In [35]:
Order=Account.send_order(code='000001',
                        price=11,
                        amount=holdnum,
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [36]:
Order

< QA_Order realorder_id Order_n0KWZa8A datetime:2018-05-10 09:31:00 code:000001 amount:3600 price:11 towards:-1 btype:stock_cn order_id:Order_n0KWZa8A account:Acc_z65rKWuO status:queued >

In [37]:
Account.cash_available  # 因为此时订单尚未申报成功 可用现金不变

960474.952

In [38]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

['Acc_z65rKWuO', '2018-05-10 09:31:00', '2018-05-10 00:00:00', '000001', None, -1, 11.0, 11.0, 'trade_success', 3600, 3600, 39669.3, 0, 'Order_n0KWZa8A', 'Trade_R2TA51pY']
< QA_Order realorder_id Order_n0KWZa8A datetime:2018-05-10 09:30:00 code:000001 amount:3600 price:11.0 towards:-1 btype:stock_cn order_id:Order_n0KWZa8A account:Acc_z65rKWuO status:queued >


In [39]:
trade_mes=B.query_orders(Account.account_cookie,'filled')
res=trade_mes.loc[Order.account_cookie,Order.realorder_id]
Order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)

receive deal


In [40]:
Account.cash_available # 此时订单已成交 cash_available立刻结转

1000144.2520000001

In [41]:
Account.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax
0,2018-05-09 00:00:00,000001,10.96,3600,960474.952,Order_y6KqNdRr,Order_y6KqNdRr,Trade_8oSy6TFV,Acc_z65rKWuO,9.864,59.184
1,2018-05-10 00:00:00,000001,11.00,-3600,1000144.252,Order_n0KWZa8A,Order_n0KWZa8A,Trade_LbO2e69C,Acc_z65rKWuO,-9.900,-59.400


In [42]:
Account.orders

<QA_ORDERQueue>

In [43]:
Account.orders.order_list

{'Order_y6KqNdRr': < QA_Order realorder_id Order_y6KqNdRr datetime:2018-05-09 09:30:00 code:000001 amount:3600 price:10.955 towards:1 btype:stock_cn order_id:Order_y6KqNdRr account:Acc_z65rKWuO status:success_all >,
 'Order_n0KWZa8A': < QA_Order realorder_id Order_n0KWZa8A datetime:2018-05-10 09:30:00 code:000001 amount:3600 price:11.0 towards:-1 btype:stock_cn order_id:Order_n0KWZa8A account:Acc_z65rKWuO status:success_all >}

# 测试T0账户

In [44]:
 # 初始化一个account
AccountT0=QA.QA_Account(running_environment=QA.RUNNING_ENVIRONMENT.TZERO,init_hold={'000001':10000},init_cash=200000)

# 初始化一个回测类
B = QA.QA_BacktestBroker()

In [45]:
AccountT0.init_assets

{'cash': 200000, 'hold': {'000001': 10000}}

In [46]:
AccountT0.init_hold

code
000001    10000
Name: amount, dtype: int64

In [47]:
AccountT0.hold_available

Series([], Name: amount, dtype: float64)

In [48]:
AccountT0.sell_available

code
000001    10000
Name: amount, dtype: int64

In [49]:
Order=AccountT0.send_order(code='000001',
                        price=11,
                        amount=AccountT0.sell_available.get('000001',0),
                        time='2018-05-10',
                        towards=QA.ORDER_DIRECTION.SELL,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_AMOUNT
                        )


In [50]:
Order.datetime

'2018-05-10 09:31:00'

In [51]:
rec_mes=B.receive_order(QA.QA_Event(order=Order))
print(rec_mes)

['Acc_n65ja9kc', '2018-05-10 09:31:00', '2018-05-10 00:00:00', '000001', None, -1, 11.0, 11.0, 'trade_success', 10000, 10000, 110192.5, 0, 'Order_uibD2Whc', 'Trade_OCYKgTSQ']
< QA_Order realorder_id Order_uibD2Whc datetime:2018-05-10 09:30:00 code:000001 amount:10000 price:11.0 towards:-1 btype:stock_cn order_id:Order_uibD2Whc account:Acc_n65ja9kc status:queued >


In [52]:
rec_mes

< QA_Order realorder_id Order_uibD2Whc datetime:2018-05-10 09:30:00 code:000001 amount:10000 price:11.0 towards:-1 btype:stock_cn order_id:Order_uibD2Whc account:Acc_n65ja9kc status:queued >

In [53]:
B.query_orders(AccountT0.account_cookie,'filled')

,,trade_time,code,name,towards,trade_amount,trade_price,trade_money,trade_id
account_cookie,realorder_id,,,,,,,,
Acc_n65ja9kc,Order_uibD2Whc,2018-05-10 00:00:00,000001,None,-1,10000,11.0,110192.5,Trade_gzxsXGkv


In [54]:
trade_mes=B.query_orders(AccountT0.account_cookie,'filled')
res=trade_mes.loc[Order.account_cookie,Order.realorder_id]
Order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)

receive deal


In [55]:
AccountT0.sell_available

code
000001    0
Name: amount, dtype: int64

In [56]:
Order.trade_time

['2018-05-09 00:00:00', '2018-05-10 00:00:00', '2018-05-10 00:00:00']

In [57]:
AccountT0.buy_available

code
000001    10000
Name: amount, dtype: int64

In [58]:
AccountT0.hold_available

code
000001   -10000
Name: amount, dtype: int64

In [59]:
AccountT0.running_time

datetime.datetime(2018, 8, 18, 0, 16, 16, 245814)

In [60]:
AccountT0.datetime

'2018-05-10 00:00:00'

In [61]:
AccountT0.buy_available.get('000001')

10000

In [62]:
r=AccountT0.close_positions_order

In [63]:
AccountT0.date

'2018-05-10'

In [64]:
for Order in r:
    print(vars(Order))
    rec_mes=B.receive_order(QA.QA_Event(order=Order))
    trade_mes=B.query_orders(AccountT0.account_cookie,'filled')
    res=trade_mes.loc[Order.account_cookie,Order.realorder_id]
    Order.trade(res.trade_id,res.trade_price,res.trade_amount,res.trade_time)

{'price': 0, 'datetime': '2018-05-10 15:00:00', 'date': '2018-05-10', 'sending_time': '2018-05-10 15:00:00', 'trade_time': ['2018-05-09 00:00:00', '2018-05-10 00:00:00', '2018-05-10 00:00:00'], 'amount': 10000, 'trade_amount': 0, 'cancel_amount': 0, 'towards': 1, 'code': '000001', 'user': None, 'market_type': 'stock_cn', 'frequence': 'day', 'account_cookie': 'Acc_n65ja9kc', 'strategy': None, 'type': 'stock_cn', 'order_model': 'CLOSE', 'amount_model': 'by_amount', 'order_id': 'Order_PhmtWjJq', 'realorder_id': 'Order_PhmtWjJq', 'commission_coeff': 0.00025, 'tax_coeff': 0.0015, 'trade_id': ['Trade_8oSy6TFV', 'Trade_LbO2e69C', 'Trade_gzxsXGkv'], 'trade_price': 0, 'callback': <bound method QA_Account.receive_deal of < QA_Account Acc_n65ja9kc>>, 'money': 0.0, 'reason': None, '_status': 'new'}
['Acc_n65ja9kc', '2018-05-10 15:00:00', '2018-05-10 00:00:00', '000001', None, 1, 11.01, 11.01, 'trade_success', 10000, 10000, 110127.525, 0, 'Order_PhmtWjJq', 'Trade_WThuZvl7']
receive deal


In [65]:
AccountT0.cash

[200000, 310192.5, 199899.825]

In [66]:
AccountT0.sell_available

code
000001    0
Name: amount, dtype: int64

In [67]:
AccountT0.hold_available

Series([], Name: amount, dtype: float64)

In [68]:
AccountT0.hold

code
000001    10000
Name: amount, dtype: int64

In [69]:
AccountT0.settle()

In [70]:
AccountT0.daily_hold

,code,000001
date,account_cookie,
2018-05-10,Acc_n65ja9kc,0


In [71]:
AccountT0.daily_cash

,,cash,datetime,date,account_cookie
date,account_cookie,,,,
2018-05-10,Acc_n65ja9kc,199900,2018-05-10 00:00:00,2018-05-10,Acc_n65ja9kc


In [72]:
AccountT0.hold_table()

code
000001    10000
Name: amount, dtype: int64

In [73]:
AccountT0.hold_price()

Series([], dtype: float64)

In [74]:
AccountT0.datetime

'2018-05-11 09:30:00'

In [75]:
AccountT0.sell_available

code
000001    10000
Name: amount, dtype: int64

In [76]:
risk_t0=QA.QA_Risk(AccountT0)

In [77]:
risk_t0.init_assets

{'cash': 200000, 'hold': {'000001': 10000}}

In [78]:
risk_t0.init_cash-risk_t0.assets

date
2018-05-10    100.175
Name: 0, dtype: float64

In [79]:
risk_t0.assets.iloc[-1]/risk_t0.init_cash-1

-0.0005008749999999562

In [80]:
risk_t0.profit_construct

{'total_buyandsell': -99.99,
 'total_tax': -0.15,
 'total_commission': -0.03,
 'total_profit': -100.17}